In [1]:
from __future__ import print_function
import tensorflow as tf
import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from siamese import SiameseNetwork

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

Using TensorFlow backend.


In [2]:
batch_size = 4
num_classes = 2
epochs = 50

# input image dimensions
img_height, img_width = 256, 256
img_rows, img_cols = 256, 256

In [3]:
pickle_in = open("/home/trojan/Desktop/dimentia/CNN_simple/kfold/X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("/home/trojan/Desktop/dimentia/CNN_simple/kfold/y.pickle","rb")
y = pickle.load(pickle_in)

In [4]:
X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)

(19677, 256, 256, 3)
(19677,)


In [5]:
x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  shuffle=True, stratify=y, 
                                                  test_size=0.2, random_state=42)

In [6]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'validation samples')
print('y_train shape:', y_train.shape)

x_train shape: (15741, 256, 256, 3)
15741 train samples
3936 validation samples
y_train shape: (15741,)


In [7]:
#print(x_train[1].dtype)
print(np.min(x_train[1]), np.max(x_train[1]))

0 1


In [8]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_val = x_val.reshape(x_val.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

In [9]:
def create_base_model(input_shape):
    model_input = Input(shape=input_shape)

    embedding = Conv2D(32, kernel_size=(5, 5), input_shape=input_shape)(model_input)
    embedding = BatchNormalization()(embedding)
    embedding = Activation(activation='relu')(embedding)
    embedding = MaxPooling2D(pool_size=(2, 2))(embedding)
    embedding = Conv2D(64, kernel_size=(3, 3))(embedding)
    embedding = BatchNormalization()(embedding)
    embedding = Activation(activation='relu')(embedding)
    embedding = MaxPooling2D(pool_size=(2, 2))(embedding)
    embedding = Conv2D(128, kernel_size=(3, 3), input_shape=input_shape)(model_input)
    embedding = BatchNormalization()(embedding)
    embedding = Activation(activation='relu')(embedding)
    embedding = MaxPooling2D(pool_size=(2, 2))(embedding)
    embedding = Conv2D(256, kernel_size=(3, 3))(embedding)
    embedding = BatchNormalization()(embedding)
    embedding = Activation(activation='relu')(embedding)
    embedding = MaxPooling2D(pool_size=(2, 2))(embedding)
    embedding = Conv2D(512, kernel_size=(3, 3))(embedding)
    embedding = BatchNormalization()(embedding)
    embedding = Activation(activation='relu')(embedding)
    embedding = MaxPooling2D(pool_size=(2, 2))(embedding)
    embedding = Flatten()(embedding)
    embedding = Dense(128)(embedding)
    embedding = BatchNormalization()(embedding)
    embedding = Activation(activation='relu')(embedding)

    return Model(model_input, embedding)

In [10]:
def create_head_model(embedding_shape):
    embedding_a = Input(shape=128)
    embedding_b = Input(shape=128)

    head = Concatenate()([embedding_a, embedding_b])
    head = Dense(8)(head)
    head = BatchNormalization()(head)
    head = Activation(activation='sigmoid')(head)

    head = Dense(1)(head)
    head = BatchNormalization()(head)
    head = Activation(activation='sigmoid')(head)

    return Model([embedding_a, embedding_b], head)

In [11]:
def lr_schedule(epoch):

    lr = 1e-3 
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 80:
        lr *= 1e-3
    elif epoch > 60: # 120
        lr *= 1e-2
    elif epoch > 30: #80
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [12]:
base_model = create_base_model(input_shape)
head_model = create_head_model(base_model.output_shape)

In [13]:
lr_scheduler = LearningRateScheduler(lr_schedule)

siamese_network = SiameseNetwork(base_model, head_model)
siamese_network.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                        optimizer=tf.keras.optimizers.Adam(lr=lr_schedule(0)),
                        metrics=[tf.keras.metrics.BinaryAccuracy()])

siamese_checkpoint_path = "./siamese_checkpoint"
filepath = siamese_checkpoint_path + '/' + 'siamese_model.h5'

Learning rate:  0.001


In [14]:
siamese_callbacks = [
    #EarlyStopping(monitor='val_accuracy', patience=10, verbose=0),
    ModelCheckpoint(filepath, monitor='val_binary_accuracy', mode='max', save_best_only=True, verbose=1),
]

In [15]:
siamese_network.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=siamese_callbacks)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
1510/3935 [==========>...................] - ETA: 4:53 - loss: 0.7143 - binary_accuracy: 0.5060

KeyboardInterrupt: 

In [ ]:
siamese_network.load_weights(filepath)
embedding = base_model.outputs[-1]

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

In [ ]:
# Add softmax layer to the pre-trained embedding network
embedding = Dense(1)(embedding)
embedding = BatchNormalization()(embedding)
embedding = Activation(activation='sigmoid')(embedding)

In [ ]:
model = Model(base_model.inputs[0], embedding)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                        optimizer=tf.keras.optimizers.Adam(lr=lr_schedule(0)),
                        metrics=[tf.keras.metrics.BinaryAccuracy()])

model_checkpoint_path = "./model_checkpoint"
filepath = model_checkpoint_path + '/' + 'best_model.h5'

In [ ]:
model__callbacks = [
    #EarlyStopping(monitor='val_accuracy', patience=10, verbose=0),
    ModelCheckpoint(filepath, monitor='val_binary_accuracy', save_best_only=True, verbose=1)
]

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=model__callbacks,
          validation_data=(x_val, y_val)
         )

In [ ]:
model.load_weights(filepath)
score = model.evaluate(x_val, y_val, verbose=0)
print('Val loss:', score[0])
print('Val accuracy:', score[1])